In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-02-05 16:15:20.088625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: This version of jaxlib was built using AVX instructions, which your CPU and/or operating system do not support. This error is frequently encountered on macOS when running an x86 Python installation on ARM hardware. In this case, try installing an ARM build of Python. Otherwise, you may be able work around this issue by building jaxlib from source.

In [ ]:
# 数据导入
user_table = pd.read_csv("user_table.csv")
search_page_table = pd.read_csv("search_page_table.csv")
payment_page_table = pd.read_csv("payment_page_table.csv")
payment_confirmation_table = pd.read_csv("payment_confirmation_table.csv")

# 数据合并
full_data = user_table.copy()
full_data['visited_search'] = full_data['user_id'].isin(search_page_table['user_id']).astype(int)
full_data['visited_payment'] = full_data['user_id'].isin(payment_page_table['user_id']).astype(int)
full_data['confirmed_payment'] = full_data['user_id'].isin(payment_confirmation_table['user_id']).astype(int)

# 创建目标变量
full_data['target'] = full_data['confirmed_payment']

# 数据预处理
label_enc = LabelEncoder()
full_data['device'] = label_enc.fit_transform(full_data['device'])
full_data['sex'] = label_enc.fit_transform(full_data['sex'])

# 特征选择与归一化
features = ['device', 'sex', 'visited_search', 'visited_payment']
scaler = MinMaxScaler()
X = scaler.fit_transform(full_data[features])
y = to_categorical(full_data['target'])



In [ ]:
# 数据分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 调整数据形状以适应 LSTM 输入
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# 构建 LSTM 模型
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# 评估模型
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')
